# Homework - 2 
***
**Name**: Connor Larson 
Worked with Rick Gentry, Joe Theis, and devin burke

***

This assignment is due on Canvas by **5pm on Friday October 2nd**. Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your classmates and instructors, but **you must write all code and solutions on your own**, and list any people or sources consulted.

## Boosting - Extra Credit [5-points]
***

In this problem, we slightly modify the AdaBoost algorithm to better explore some properties of the algorithm. Specifically, we no longer normalize the weights on the training examples after each iteration. The modified algorithm, which is set to run for $T$ iterations, is shown in Algorithm I.

Note that in the modified version, the weights associated with the training examples are no longer guaranteed to sum to one after each iteration (and therefore can not be viewed as a "distribution"), but the algorithm is still valid. Let us denote the sum of weights at the start of iteration $t$ by $Z_t = \sum_{i=1}^{n}w_t^{(t)}$. At the start of the first iteration of boosting, $Z_1 = n$. Let us now investigate the behavior of $Z_t$, as a function of t

![image](fig-1.png)

**A:** At the $i^{th}$ iteration, we found a weak classifier that achieves a weighted training error $\epsilon_t$. Show that the choice, $\alpha_t = \frac{1}{2}\log\frac{1 - \epsilon_t}{\epsilon_t}$ is the optimal in the sense that it minimizes $Z_{t+1}$

*Hint: Look at $Z_{t+1}$ as function of $\alpha$ and find the value of $\alpha$ for which the function achieves the maximum. You may also find the following notational shorthand useful:

$$W_t = \sum_{i=1}^{n}w_i^{(t)}(1 - \delta(y_i, h_t(x_i)))$$
$$W_c = \sum_{i=1}^{n}w_i^{(t)}(\delta(y_i, h_t(x_i)))$$

where $W_c$ is the total weight of the points classified correctly by $h_t$ and $W_t$ is the total weight of the misclassified points. $\delta(y, h_t(x)) = 1$ whenever the label predicted by $h_t$ is correct and zero otherwise. The weights here are those available at the start of iteration $t$

From the hint we see that for a point that is correct classified the new wieght formula becomes:
$w_i^{(t)}\exp(\alpha_t)$

And for a point that is incorectly classified the new wieght formula is: 
$w_i^{(t)}\exp(-\alpha_t)$

Thus the new sum for $Z_{t+1}(\alpha_t)= W_c + W_t$  becomes $ w_c\exp(\alpha_t)+w_t\exp(-\alpha_t)$
    
Taking the deravative of that with respect to $\alpha_t$ with the formula set to 0 : 
    $ \frac{\delta Z_{t+1}}{\delta \alpha_t} = w_c\exp(\alpha_t)-w_t\exp(-\alpha_t)=0$ 

=> $\exp(2 \alpha_t) = \frac{W_c}{W_t}$  

=> $\alpha_t = .5 * log( \frac{W_c}{W_t} )$

and by definition $W_c = Z_t*(1 - \epsilon_t )$ and $ W_t= Z_t* \epsilon_t $

thus we have $\alpha_t = \frac{1}{2}\log\frac{1 - \epsilon_t}{\epsilon_t}$



**B:** Show that the sum of weights $Z_t$ is monotonically decreasing as a function of $t$.

## Training Data
***
Please do not change this class

In [5]:
import numpy as np
from sklearn.base import clone

In [6]:
class ThreesAndEights:
    """
    Class to store MNIST data
    """

    def __init__(self, location):

        import pickle, gzip

        # Load the dataset
        f = gzip.open(location, 'rb')

        # Split the data set 
#         X_train, y_train, X_valid, y_valid = pickle.load(f)
        train_set, valid_set, test_set = pickle.load(f)
    
        X_train, y_train = train_set
        X_valid, y_valid = valid_set

        # Extract only 3's and 8's for training set 
        self.X_train = X_train[np.logical_or( y_train==3, y_train == 8), :]
        self.y_train = y_train[np.logical_or( y_train==3, y_train == 8)]
        self.y_train = np.array([1 if y == 8 else -1 for y in self.y_train])
        
        # Shuffle the training data 
        shuff = np.arange(self.X_train.shape[0])
        np.random.shuffle(shuff)
        self.X_train = self.X_train[shuff,:]
        self.y_train = self.y_train[shuff]

        # Extract only 3's and 8's for validation set 
        self.X_valid = X_valid[np.logical_or( y_valid==3, y_valid == 8), :]
        self.y_valid = y_valid[np.logical_or( y_valid==3, y_valid == 8)]
        self.y_valid = np.array([1 if y == 8 else -1 for y in self.y_valid])
        
        f.close()

In [7]:
data = ThreesAndEights("data/mnist.pklz")

Feel free to explore this data and get comfortable with it before proceeding further.

## Bagging
Bootstrap aggregating, also called bagging, is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting. Although it is usually applied to decision tree methods, it can be used with any type of method. Bagging is a special case of the model averaging approach.

Given a standard training set $D$ of size n, bagging generates $N$ new training sets $D_i$, roughly each of size n * ratio, by sampling from $D$ uniformly and with replacement. By sampling with replacement, some observations may be repeated in each $D_i$ The $N$ models are fitted using the above $N$ bootstraped samples and combined by averaging the output (for regression) or voting (for classification). 

-Source [Wiki](https://en.wikipedia.org/wiki/Bootstrap_aggregating)

## Implementing Bagging [5-points]
***

We've given you a skeleton of the class `BaggingClassifier` below which will train a classifier based on the decision trees as implemented by sklearn. Your tasks are as follows, please approach step by step to understand the code flow:
* Implement `bootstrap` method which takes in two parameters (`X_train, y_train`) and returns a bootstrapped training set ($D_i$)
* Implement `fit` method which takes in two parameters (`X_train, y_train`) and trains `N` number of base models on different bootstrap samples. You should call `bootstrap` method to get bootstrapped training data for each of your base model
* Implement `voting` method which takes the predictions from learner trained on bootstrapped data points `y_hats` and returns final prediction as per majority rule. In case of ties, return either of the class randomly.
* Implement `predict` method which takes in multiple data points and returns final prediction for each one of those. Please use the `voting` method to reach consensus on final prediction.

In [153]:
from sklearn.tree import DecisionTreeClassifier

class BaggingClassifier:
    def __init__(self, ratio = 0.73, N = 20, base=DecisionTreeClassifier(max_depth=4)):
        """
        Create a new BaggingClassifier
        
        Args:
            base (BaseEstimator, optional): Sklearn implementation of decision tree
            ratio: ratio of number of data points in subsampled data to the actual training data
            N: number of base estimator in the ensemble
        
        Attributes:
            base (estimator): Sklearn implementation of decision tree
            N: Number of decision trees
            learners: List of models trained on bootstrapped data sample
        """
        
        assert ratio <= 1.0, "Cannot have ratio greater than one"
        self.base = base
        self.ratio = ratio
        self.N = N
        self.learners = []
        
    def fit(self, X_train, y_train):
        """
        Train Bagging Ensemble Classifier on data
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        #TODO: Implement functionality to fit models on the bootstrapped samples
        # cloning sklearn models:
        # from sklearn.base import clone
        # h = clone(self.base)
        for x in range(self.N):
            h = clone(self.base)
            new_X, new_y = self.bootstrap(X_train, y_train)
            h = h.fit(new_X,new_y)
            self.learners.append(h)
        
    def bootstrap(self, X_train, y_train):
        """
        Args:
            n (int): total size of the training data
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        indexes = np.random.choice(X_train.shape[0], int(self.ratio * X_train.shape[0]))
        return X_train[indexes], y_train[indexes]
            
    
    def predict(self, X):
        """
        BaggingClassifier prediction for data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        #TODO: Using the individual classifiers trained predict the final prediction using voting mechanism
        yhat = np.zeros(len(self.learners))
        for x in range(len(self.learners)):
            yhat[x]= self.learners[x].predict(X)
        
        final_predict= self.voting(yhat)
        return final_predict
        
    def voting(self, y_hats):
        """
        Args:
            y_hats (ndarray): [N] ndarray of data
        Returns:
            y_final : int, final prediction of the 
        """
        #TODO: Implement majority voting scheme and incase of ties return random label
        unique, counts = np.unique(y_hats, return_counts=True)
        count_dict= dict(zip(unique, counts))
        if (len(unique) == 1):
            return unique[0]
        else : 
            if (count_dict[1]> count_dict[-1]):
                return 1
            elif (count_dict[-1]> count_dict[1]):
                return -1 
            elif (count_dict[1] == count_dict[-1]):
                rand_label = np.random.choice([-1,1], 1)
                return rand_label[0]
        

## BaggingClassifier for Handwritten Digit Recognition [5-points]
***

After you've successfully completed `BaggingClassifier` find the optimal values of `N` and `depth` using k-fold cross validation. You are allowed to use sklearn library to split your training data in folds. Keep the other hyperparameters unchanged. Use the data from `ThreesAndEights` class initialized variable `data`. 

Justify why those values are optimal

Report accuracy on the validation data using the optimal parameter values.

What is the most deciding hyperparameter and why?

Hint:  Vary `depth` up to 10, `N` up to 40. The number of decision trees `N` is generally a trade-off between 'improvement in accuracy' vs 'computation time'.

In [155]:
#ratio = 0.73, N = 20, base=DecisionTreeClassifier(max_depth=4)
from sklearn.model_selection import KFold
depth_loop =[4,6,8,10]
N_loop =[20,24,28,32,36,40]
for depth in depth_loop:
    for N_n in N_loop:
        print('depth:', depth, 'N:', N_n)
        kf = KFold(n_splits=3)
        kf.get_n_splits(data.X_train)
        correct_list=[]
        for train_index, test_index in kf.split(data.X_train):
            x_tr, x_tes = data.X_train[train_index], data.X_train[test_index]
            y_tr, y_tes = data.y_train[train_index], data.y_train[test_index]
            BC= BaggingClassifier(ratio = 0.73, N = N_n, base=DecisionTreeClassifier(max_depth=depth))
            BC.fit(x_tr, y_tr)
            correct=0
            for index in range(x_tes.shape[0]):
                prediction = BC.predict([x_tes[index]])
                if prediction == y_tes[index]:
                    correct+=1
            correct_list.append(correct/y_tes.shape[0])
        total=0
        for item in correct_list:
            total+= item
        print(total/3)

depth: 4 N: 20
0.9537364375519793
depth: 4 N: 24
0.9525294066065836
depth: 4 N: 28
0.9512221867222044
depth: 4 N: 32
0.9525294976322702
depth: 4 N: 36
0.952026641397936
depth: 4 N: 40
0.9519262097237885
depth: 6 N: 20
0.9627882745565305
depth: 6 N: 24
0.9631906384329261
depth: 6 N: 28
0.9649001311680143
depth: 6 N: 32
0.963291100448969
depth: 6 N: 36
0.962989289614303
depth: 6 N: 40
0.9643974569850533
depth: 8 N: 20
0.965805229911162
depth: 8 N: 24
0.9668110334055168
depth: 8 N: 28
0.9661069800620371
depth: 8 N: 32
0.9678167155322894
depth: 8 N: 36
0.9693253752609783
depth: 8 N: 40
0.9675150260652053
depth: 10 N: 20
0.968118313973687
depth: 10 N: 24
0.9691240264423552
depth: 10 N: 28
0.9697275874278963
depth: 10 N: 32
0.9695264510025418
depth: 10 N: 36
0.9686212915756031
depth: 10 N: 40
0.9691240871261462


In [174]:
kf = KFold(n_splits=3)
kf.get_n_splits(data.X_train)
for train_index, test_index in kf.split(data.X_train):
    x_tr, x_tes = data.X_train[train_index], data.X_train[test_index]
    y_tr, y_tes = data.y_train[train_index], data.y_train[test_index]
    BC= BaggingClassifier(ratio = 0.73, N = 28, base=DecisionTreeClassifier(max_depth=10))
    BC.fit(x_tr, y_tr)
    correct=0
    for index in range(x_tes.shape[0]):
        prediction = BC.predict([x_tes[index]])
        if prediction == y_tes[index]:
            correct+=1
    print(correct/y_tes.shape[0])

0.9704374057315234
0.9704284852142426
0.9671092335546168


In [175]:
correct=0
for index in range(data.X_valid.shape[0]):
    prediction = BC.predict([data.X_valid[index]])
    if prediction == data.y_valid[index]:
        correct+=1
print(correct/data.y_valid.shape[0])


0.9671407552721922


Justify why those values are optimal:

The optimal values are running several iterations with different hyper parameters is N=28 and Depth = 10. I came to this conclusion by looking at the accuracies that were reported from my Kfold runnings of my Algorithm. At N=28 and Depth = 10 my algorithm gets the highest accuracy and then starts to decline as N gets higher. This is where the Knee in the curve of errors is and therefore is the optimal parameter. 

Report accuracy on the validation data using the optimal parameter values: 

0.9671407552721922 is the accuracy on the validation set. 


What is the most deciding hyperparameter and why?:

The most deciding hyper parameter is N. This is becuase with more treess to train on the algorithm has more trees to make a decsion on. 

# Random Decision Tree [10-points]

In this assignment you are going to implement a random decision tree using random vector method as discussed in the lecture.

Best split: One that achieves maximum reduction in gini index across multiple candidate splits. (decided by `candidate_splits` attribute of the class `RandomDecisionTree`)

Use `TreeNode` class as node abstraction to build the tree

You are allowed to add new attributes in the `TreeNode` and `RandomDecisionTree` class - if that helps.

Your tasks are as follows:
* Implement `gini_index` method which takes in class labels as parameter and returns the gini impurity as measure of uncertainty

* Implement `majority` method which picks the most frequent class label. In case of tie return any random class label

* Implement `find_best_split` method which finds the random vector/hyperplane which causes most reduction in the gini index. 

* Implement `build_tree` method which uses `find_best_split` method to get the best random split vector for current set of training points. This vector partitions the training points into two sets, and you should call `build_tree` method on two partitioned sets and build left subtree and right subtree. Use `TreeNode` as abstraction for a node.

> The method calls itself recursively to the generate left and right subtree till the point either `max_depth` is reached or no good random split is found.  When either of two cases is encountered, you should make that node as leaf and identify the label for that leaf to be the most frequent class (use `majority` method). Go through lecture slides for better understanding

* Implement `predict` method which takes in multiple data points and returns final prediction for each one of those using the tree built. (`root` attribute of the class)

In [147]:
import math 
class TreeNode:
    def __init__(self):
        self.left = None
        self.right = None
        self.isLeaf = False
        self.label = None
        self.split_vector = None

    def getLabel(self):
        if not self.isLeaf:
            raise Exception("Should not do getLabel on a non-leaf node")
        return self.label
    
class RandomDecisionTree:
            
    def __init__(self, candidate_splits = 100, depth = 10):
        """
        Args:
            candidate_splits (int) : number of random decision splits to test
            depth (int) : maximum depth of the random decision tree
        """
        self.candidate_splits = candidate_splits
        self.depth = depth
        self.root = None
    
    def fit(self, X_train, y_train):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
            
        """
        self.root = self.build_tree(X_train[:], y_train[:], 0)
        return self
        
    def build_tree(self, X_train, y_train, height, node = TreeNode()):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
            
       Return:
            split_vector: random vector which gives most reduction in uncertainty
            feature_indices: indices of the random sub-features used
            lindices: indices of training example which should be in left subtree
            rindices: indices of training example which should be in right subtree
            
        """
        
        # your logic here
        if self.gini_index(y_train) ==0: 
            # if data is pure 
#             print("pure data found")
            node.isLeaf = True 
            node.label = y_train[0] # since all the values are the same, I can pick the first value 
            return node

        elif height == self.depth : 
            # if max height is reached
#             print("Max height reached")
            node.isLeaf= True
            node.label= self.majority(y_train)
            return node
        else:
#             print("splitting: height", height)
            
            split_vector =self.find_best_split(X_train, y_train)
            values = np.dot(X_train,split_vector)
            left = np.where(values<0)
            right = np.where(values>0)
            
            node.split_vector=split_vector
#             print("splitting Left")
            node.left = self.build_tree(X_train[left[0]], y_train[left[0]],height+1, node = TreeNode())
#             print("splitting Right")
            node.right = self.build_tree(X_train[right[0]], y_train[right[0]],height+1, node = TreeNode())
            return node
        
    def find_best_split(self, X_train, y_train):
        """
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data
        returns: 
            split_vector
        """
        min_gini = 1
        split_vector=None

        for x in range(RD.candidate_splits):
            potential_split_vals = np.random.randn(X_train.shape[1])
            values = np.dot(X_train,potential_split_vals)
            left = np.where(values<0)
            right = np.where(values>0)
            left_score = RD.gini_index(y_train[left[0]])
            right_score = RD.gini_index(y_train[right[0]])
            total_score = (left_score * (len(left[0])/y_train.shape[0])) + (right_score * (len(right[0])/y_train.shape[0]))
            if total_score < min_gini:
                min_gini = total_score
                split_vector=potential_split_vals
#         print('min',min_gini)
        return split_vector
            
        
    def gini_index(self, y):
        """
        Args:
            y (ndarray): [n_samples] ndarray of data
        """
        unique, counts = np.unique(y, return_counts=True)
        count_dict= dict(zip(unique, counts))
        if (len(unique) < 2):
            return 0
        else: 
            
            ratio = count_dict[1]/y.shape[0]
            return (2*(ratio) *(1-ratio))
            
    
    def majority(self, y):
        """
        Return the major class in ndarray y
        """
        unique, counts = np.unique(y, return_counts=True)
        count_dict= dict(zip(unique, counts))
        if (len(unique) == 1):
            return unique[0]
        else : 
            if (count_dict[1]> count_dict[-1]):
                return 1
            elif (count_dict[-1]> count_dict[1]):
                return -1 
            elif (count_dict[1] == count_dict[-1]):
                rand_label = np.random.choice([-1,1], 1)
                return rand_label[0]
    
    def predict(self, X):
        """
        BaggingClassifier prediction for new data points in X
        
        Args:
            X (ndarray): [n_samples x n_features] ndarray of data 
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        yhat=[]
        for item in X:
            yhat.append(self.iterate_to_leaf(self.root, item))
        
        return np.array(yhat)
            
    def iterate_to_leaf(self, node,  X):
        if node.isLeaf:
            return node.getLabel()
        else:
            value = np.dot(X, node.split_vector)
            if value < 0:
                return self.iterate_to_leaf(node.left, X)
            else: 
                return self.iterate_to_leaf(node.right, X)
        

## RandomDecisionTree for Handwritten Digit Recognition

a) After you've successfully completed `RandomDecisionTree`, and train using the default values in the constructor and report accuracy on the `valid_set`. Use the data from `ThreesAndEights` class initialized variable `data` 

b) Vary the `depth` up to 20 and comment on the trend that you observe in the accuracy scores of the `valid_set`. Base your comments on the concepts taught in the class. Keep the other hyperparameters unchanged.

In [176]:
RD = RandomDecisionTree()
RD.fit(data.X_train, data.y_train)

In [177]:
predictions = RD.predict(data.X_valid)
correct =0
for x in range(predictions.shape[0]):
    if predictions[x] == data.y_valid[x]:
        correct+=1 
print(correct / data.y_valid.shape[0] )

0.9073075036782736


In [187]:
depth_loop =[10,11,13,15,18,20]
for depth_x in depth_loop:
    RD = RandomDecisionTree(depth=depth_x)
    RD.fit(data.X_train, data.y_train)
    predictions = RD.predict(data.X_valid)

    correct =0
    for x in range(predictions.shape[0]):
        if predictions[x] == data.y_valid[x]:
            correct+=1 
    print(correct / data.y_valid.shape[0] )

0.9097596861206474
0.902893575282001
0.8827856792545365
0.8881804806277587
0.9014222658165768
0.889651790093183


By varying the dept up to 20, each time the percentage of accuracy goes down on the validation set. This is most likely cuased due to over fitting on the training data cuasing the model to not perform as well on the validation data that it has not seen before. 

# Random Forest [5-points]
Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random decision forests correct for decision trees' habit of overfitting to their training set.

Random forest trains random decision trees on bootstrapped training points. Thus, you can try implementation of methods (`bootstrap`, `predict`) from `BaggingClassifier` class directly. Only difference being, you have to use the `RandomDecisionTree` as base which you implemented previously instead of sklearn's implementation of `DecisionTreeClassifier`). Implement the `fit` method in the class below accordingly.

In [162]:
class RandomForest():
    def __init__(self, ratio = 0.63, N = 20, max_depth = 10, candidate_splits = 500):
        self.ratio = ratio
        self.N = N  
        self.learners = []
        self.candidate_splits = candidate_splits
        self.max_depth = max_depth
        
    def fit(self, X_train, y_train):
        """
        Train Bagging Ensemble Classifier on data
        
        Args:
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        for x in range(self.N):
            RD = RandomDecisionTree(depth=self.max_depth)
            new_X, new_y = self.bootstrap(X_train, y_train)
            RD = RD.fit(new_X,new_y)
            self.learners.append(RD)
            
    def bootstrap(self, X_train, y_train):
        """
        Args:
            n (int): total size of the training data
            X_train (ndarray): [n_samples x n_features] ndarray of training data   
            y_train (ndarray): [n_samples] ndarray of data 
        """
        indexes = np.random.choice(X_train.shape[0], int(self.ratio * X_train.shape[0]))
        return X_train[indexes], y_train[indexes]
            
    
   
    
    def predict(self, X):
        """
        Random Forest prediction for data points in X
        
        Args:
            X (ndarray): [n_features] ndarray of data 
            
        Returns:
            yhat (ndarray): [n_samples] ndarray of predicted labels {-1,1}
        """
        #TODO: Using the individual classifiers trained predict the final prediction using voting mechanism
        yhat = np.zeros(len(self.learners))
        for x in range(len(self.learners)):
            yhat[x]= self.learners[x].iterate_to_leaf(self.learners[x].root, X)
        
        final_predict= self.voting(yhat)
        return final_predict
        
    def voting(self, y_hats):
        """
        Args:
            y_hats (ndarray): [N] ndarray of data
        Returns:
            y_final : int, final prediction of the 
        """
        #TODO: Implement majority voting scheme and incase of ties return random label
        unique, counts = np.unique(y_hats, return_counts=True)
        count_dict= dict(zip(unique, counts))
        if (len(unique) == 1):
            return unique[0]
        else : 
            if (count_dict[1]> count_dict[-1]):
                return 1
            elif (count_dict[-1]> count_dict[1]):
                return -1 
            elif (count_dict[1] == count_dict[-1]):
                rand_label = np.random.choice([-1,1], 1)
                return rand_label[0]
        

## RandomForest for Handwritten Digit Recognition [5-points]
***

After you've successfully completed `RandomForest` find the optimal values of `N` and `max_depth` ,  using k-fold cross validation. Fix the values of the other hyperparameters to the given defaults. Feel free to use sklearn library to split your training data. Use the data from `ThreesAndEights` class intialized variable `data`. 

Justify why those values are optimal. 

Report best accuracy on the testing data using the optimal `N`.

Hint: Vary `N` up to 25 and set `max_depth` up to 10. Plan ahead as it might take some time.

In [163]:
kf = KFold(n_splits=3)
kf.get_n_splits(data.X_train)
for train_index, test_index in kf.split(data.X_train):
    x_tr, x_tes = data.X_train[train_index], data.X_train[test_index]
    y_tr, y_tes = data.y_train[train_index], data.y_train[test_index]
    RF = RandomForest()
    RF.fit(x_tr, y_tr)
    correct=0
    for index in range(x_tes.shape[0]):
        prediction = RF.predict(x_tes[index])
        if prediction == y_tes[index]:
            correct+=1
    print(correct/y_tes.shape[0])

0.8555052790346908
0.8780929390464696
0.8624019312009656


In [165]:
depth_loop =[4,6,8,10]
N_loop =[15,18,20,21,22,23,25]
for depth in depth_loop:
    for N_n in N_loop:
        print('depth:', depth, 'N:', N_n)
        kf = KFold(n_splits=3)
        kf.get_n_splits(data.X_train)
        correct_list=[]
        for train_index, test_index in kf.split(data.X_train):
            x_tr, x_tes = data.X_train[train_index], data.X_train[test_index]
            y_tr, y_tes = data.y_train[train_index], data.y_train[test_index]
            RF= RandomForest(N = N_n, max_depth=depth)
            RF.fit(x_tr, y_tr)
            correct=0
            for index in range(x_tes.shape[0]):
                prediction = RF.predict(x_tes[index])
                if prediction == y_tes[index]:
                    correct+=1
            correct_list.append(correct/y_tes.shape[0])
        total=0
        for item in correct_list:
            total+= item
        print(total/3)

depth: 4 N: 15
0.8431041215520607
depth: 4 N: 18
0.837273046414301
depth: 4 N: 20
0.8202760930440295
depth: 4 N: 21
0.8285230202444162
depth: 4 N: 22
0.8420999565200638
depth: 4 N: 23
0.84934192979917
depth: 4 N: 25
0.833450574417595
depth: 6 N: 15
0.8583929779144377
depth: 6 N: 18
0.8581899299496657
depth: 6 N: 20
0.8493416870640059
depth: 6 N: 21
0.8571859469690418
depth: 6 N: 22
0.8599013038822759
depth: 6 N: 23
0.8609060757521831
depth: 6 N: 25
0.8612097374424756
depth: 8 N: 15
0.873780020650694
depth: 8 N: 18
0.8651313060699266
depth: 8 N: 20
0.8738805130086326
depth: 8 N: 21
0.8731763382975708
depth: 8 N: 22
0.8702610282929073
depth: 8 N: 23
0.8713670207262454
depth: 8 N: 25
0.85598003867378
depth: 10 N: 15
0.8676460423700297
depth: 10 N: 18
0.8779043338239618
depth: 10 N: 20
0.8539678248471603
depth: 10 N: 21
0.8636217967681633
depth: 10 N: 22
0.8626178137875393
depth: 10 N: 23
0.8631202148934408
depth: 10 N: 25
0.8675452162512406


Justify why those values are optimal: 
    
The optimal hyper parameters are depth: 10 N: 18. By looking at all of the accuracy scores reported from running the Algorithm with different hyperparameters. These two values return the highest accuract score. It also represents the knee in the curve of errors becuase after those two values the accuracy starts to go down. 


In [182]:
RF= RandomForest(N = 18, max_depth=10)
RF.fit(data.X_train, data.y_train)

correct =0
for index in range(data.X_valid.shape[0]):
    prediction = RF.predict(data.X_valid[index])
    if prediction == data.y_valid[index]:
        correct+=1

print(correct / data.y_valid.shape[0] )

0.9087788131436979


Report best accuracy on the testing data using the optimal `N`.:

The best Accuracy on the validation data is 0.9087788131436979. This is higher than the Algorithm achieved on the training data. 